In [31]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [32]:
df = pd.read_csv('traindata2024-06-27.csv')
df.head()

,redCorner,blueCorner,winner,referee,venue,title_fight,billing,redCorner_wins,blueCorner_wins,redCorner_losses,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,0,146,135,0,0,25,15,7,...,1,7.706422,2.752294,1.816667,2.752294,7.706422,0.666667,0.333333,1.0,1.0
1,0,2,2,131,135,0,1,37,18,10,...,0,3.066667,5.533333,15.000000,5.533333,3.066667,0.477987,0.656716,1.0,1.0
2,0,2,0,190,135,0,2,18,17,9,...,1,7.466667,8.466667,15.000000,8.466667,7.466667,0.520755,0.529412,1.0,0.0
3,0,2,0,146,135,0,2,12,12,0,...,0,6.586345,1.285141,12.450000,1.285141,6.586345,0.529412,0.240741,1.0,1.0
4,0,2,2,131,135,0,2,19,21,7,...,0,6.081081,15.000000,2.466667,15.000000,6.081081,0.447761,0.545455,1.0,1.0


In [33]:
dfInput = pd.read_csv('customInput.csv')
dfInput = dfInput.drop('Unnamed: 0', axis=1, errors='ignore')
dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,redCorner_fightTime,blueCorner_fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,Dan Ige,Diego Lopes,NaN,18.0,24.0,7.0,6.0,0,0,32.0,...,3.76,3.22,12.516667,5.966667,3.52,4.73,0.57,0.37,0.58,0.42


In [34]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6379
6379


In [35]:
#dropping referee
df.drop('referee', axis=1, inplace=True)

#drop billings
df.drop('billing', axis=1, inplace=True)

#drop venue
df.drop('venue', axis=1, inplace=True)

#drop title_fight
df.drop('title_fight', axis=1, inplace=True)


df.head()


,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,0,25,15,7,1,0,0,33.0,...,1,7.706422,2.752294,1.816667,2.752294,7.706422,0.666667,0.333333,1.0,1.0
1,0,2,2,37,18,10,2,0,0,35.0,...,0,3.066667,5.533333,15.000000,5.533333,3.066667,0.477987,0.656716,1.0,1.0
2,0,2,0,18,17,9,4,0,0,32.0,...,1,7.466667,8.466667,15.000000,8.466667,7.466667,0.520755,0.529412,1.0,0.0
3,0,2,0,12,12,0,3,0,0,30.0,...,0,6.586345,1.285141,12.450000,1.285141,6.586345,0.529412,0.240741,1.0,1.0
4,0,2,2,19,21,7,8,0,0,34.0,...,0,6.081081,15.000000,2.466667,15.000000,6.081081,0.447761,0.545455,1.0,1.0


In [36]:
df.rename(columns={'fightTime': 'redCorner_fightTime'}, inplace=True)
df.insert(32, 'blueCorner_fightTime', df['redCorner_fightTime'])

dfColumns = df.columns
print(len(dfColumns))

39


In [37]:
dfInputColumns = dfInput.columns
print(len(dfInputColumns))

39


In [38]:
#one hot encode redCorner_nation
column = df[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = df[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_101,blueCorner_nation_103,blueCorner_nation_104,blueCorner_nation_105,blueCorner_nation_106,blueCorner_nation_107,blueCorner_nation_108,blueCorner_nation_110,blueCorner_nation_111,blueCorner_nation_112
0,0,2,0,25,15,7,1,0,0,33.0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,2,37,18,10,2,0,0,35.0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,18,17,9,4,0,0,32.0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,12,12,0,3,0,0,30.0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,2,19,21,7,8,0,0,34.0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
#one hot encode redCorner_stance
column = df[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = df[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,2,0,25,15,7,1,0,0,33.0,...,0,0,1,0,0,0,0,1,0,0
1,0,2,2,37,18,10,2,0,0,35.0,...,0,0,0,1,0,0,1,0,0,0
2,0,2,0,18,17,9,4,0,0,32.0,...,0,0,0,1,0,0,0,1,0,0
3,0,2,0,12,12,0,3,0,0,30.0,...,0,0,1,0,0,0,1,0,0,0
4,0,2,2,19,21,7,8,0,0,34.0,...,0,0,1,0,0,0,1,0,0,0


In [40]:
for index, row in df.iterrows():
    if row['winner'] == 1:
        df.drop(index)

df['blueCorner'] = 1
df['winner'] = df['winner'].replace(2, 1)


df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,1,0,25,15,7,1,0,0,33.0,...,0,0,1,0,0,0,0,1,0,0
1,0,1,1,37,18,10,2,0,0,35.0,...,0,0,0,1,0,0,1,0,0,0
2,0,1,0,18,17,9,4,0,0,32.0,...,0,0,0,1,0,0,0,1,0,0
3,0,1,0,12,12,0,3,0,0,30.0,...,0,0,1,0,0,0,1,0,0,0
4,0,1,1,19,21,7,8,0,0,34.0,...,0,0,1,0,0,0,1,0,0,0


In [41]:
#one hot encode redCorner_nation
column = df[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
dfInput.drop('redCorner_nation', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = df[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
dfInput.drop('blueCorner_nation', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

df.drop('redCorner_nation', axis=1, inplace=True)
df.drop('blueCorner_nation', axis=1, inplace=True)

dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_101,blueCorner_nation_103,blueCorner_nation_104,blueCorner_nation_105,blueCorner_nation_106,blueCorner_nation_107,blueCorner_nation_108,blueCorner_nation_110,blueCorner_nation_111,blueCorner_nation_112
0,Dan Ige,Diego Lopes,NaN,18.0,24.0,7.0,6.0,0.0,0.0,32.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [42]:
#one hot encode redCorner_stance
column = df[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
dfInput.drop('redCorner_stance', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = df[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
dfInput.drop('blueCorner_stance', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

df.drop('redCorner_stance', axis=1, inplace=True)
df.drop('blueCorner_stance', axis=1, inplace=True)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,1,0,25,15,7,1,0,0,33.0,...,0,0,1,0,0,0,0,1,0,0
1,0,1,1,37,18,10,2,0,0,35.0,...,0,0,0,1,0,0,1,0,0,0
2,0,1,0,18,17,9,4,0,0,32.0,...,0,0,0,1,0,0,0,1,0,0
3,0,1,0,12,12,0,3,0,0,30.0,...,0,0,1,0,0,0,1,0,0,0
4,0,1,1,19,21,7,8,0,0,34.0,...,0,0,1,0,0,0,1,0,0,0


In [43]:
#limit the data
dfInput = dfInput.head(1)
print(dfInput.iloc[0])

redCorner                  Dan Ige
blueCorner             Diego Lopes
winner                         NaN
redCorner_wins                18.0
blueCorner_wins               24.0
                          ...     
redCorner_stance_3               0
blueCorner_stance_0              0
blueCorner_stance_1              1
blueCorner_stance_2              0
blueCorner_stance_3              0
Name: 0, Length: 252, dtype: object


In [44]:
dfColumns = df.columns
for column in dfColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
redCorner_fightTime
blueCorner_fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0
redCorner_nation_1
redCorner_nation_2
redCorner_nation_3
redCorner_nation_4
redCorner_nation_5
redCorner_nation_6
redCorner_nation_7
redCorner_nation_8
redCorner_nation_9
redCorner_nation_10
redCorner_na

In [45]:
dfInputColumns = dfInput.columns
for column in dfInputColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
redCorner_fightTime
blueCorner_fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0
redCorner_nation_1
redCorner_nation_2
redCorner_nation_3
redCorner_nation_4
redCorner_nation_5
redCorner_nation_6
redCorner_nation_7
redCorner_nation_8
redCorner_nation_9
redCorner_nation_10
redCorner_na

In [46]:
#when one hot encoding, nations that have no representation in the train set have Nan values. This sets them to zero as they would have been zero if they were represented
df.fillna(0, inplace=True)

In [47]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6379
6379


In [48]:
print(len(dfInput))
dfInput.dropna(subset=dfInput.columns.difference(['winner']), inplace=True)
print(len(dfInput))

1
1


In [49]:
dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,Dan Ige,Diego Lopes,NaN,18.0,24.0,7.0,6.0,0.0,0.0,32.0,...,0,0,1,0,0,0,0,1,0,0


In [50]:
#x,y split
target_column = 'winner'
y = df[target_column]
X = df.drop(target_column, axis=1)

#create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





In [51]:
#Define the deep learning model
model = Sequential()
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

#earlyStopping
earlystopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath='best_model_weights2.weights.h5', save_weights_only=True, monitor='val_accuracy', save_best_only=True, verbose=1)

#compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train
model.fit(X_train, y_train, epochs=1000, batch_size=124, validation_data=(X_test, y_test), callbacks=[checkpoint, earlystopping])

Epoch 1/1000
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6190 - loss: 0.6443
Epoch 1: val_accuracy improved from -inf to 0.79075, saving model to best_model_weights2.weights.h5
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.6207 - loss: 0.6428 - val_accuracy: 0.7908 - val_loss: 0.4968
Epoch 2/1000
41/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7834 - loss: 0.4669
Epoch 2: val_accuracy did not improve from 0.79075
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7837 - loss: 0.4661 - val_accuracy: 0.7618 - val_loss: 0.4804
Epoch 3/1000
35/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8077 - loss: 0.4151
Epoch 3: val_accuracy improved from 0.79075 to 0.83777, saving model to best_model_weights2.weights.h5
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8123 - loss: 0.4106 - val_accuracy: 0.8378 - val_loss: 0.3965
Epoch 4/1000
34/42 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8472 - loss: 0.3585
Epoch 4: val_accuracy did not improve from 0.83777
42

In [52]:
for index, row in dfInput.iterrows():
    dfInput.at[index, 'redCorner'] = 0.0
    dfInput.at[index, 'blueCorner'] = 1.0

dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0.0,1.0,NaN,18.0,24.0,7.0,6.0,0.0,0.0,32.0,...,0,0,1,0,0,0,0,1,0,0


In [53]:
dfInput = dfInput.astype(float)

In [54]:
#accuracy
y_hat = model.predict(X_test)
y_hat = [0 if val<0.5 else 1 for val in y_hat]
accuracy_score(y_test,y_hat)

#0.8545897644191714

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


0.8581504702194357

In [55]:
#testing the model using career statistics

# x, y split for dfMock
target_column = 'winner'
y_mock = dfInput[target_column]
X_mock = dfInput.drop(target_column, axis=1)

#accuracy
y_hat = model.predict(X_mock)
print(y_hat)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.9931621]]
